# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [1]:
!pip install datasets

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [3]:
from datasets import load_dataset

# Carga del dataset con confianza en el código remoto
dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl", trust_remote_code=True) # or razones de seguridad, la biblioteca datasets no ejecuta ese código automáticamente a menos que explícitamente le indiques que confías en él.  `trust_remote_code=True`

# Ver las primeras filas del dataset
print(dataset_corpus)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})


In [4]:
dataset_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

In [5]:
subset = dataset_corpus['train'].select(range(1000000))

In [6]:
subset[0:2]

{'text': ['lavado de cerebro a través de los medios de comunicación, y amenaza de fuerza a través de los militares.',
  'Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.']}

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\devel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Pre-procesamiento de texto

In [8]:
def clean_text(sentence_batch):
    # extrae el texto de la entrada
    text_list = sentence_batch['text']

    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Elimina las menciones @ y '#' de las redes sociales
        text = re.sub(r'\@\w+|\#\w+', '', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text,minsize=2)

        # Elimina las palabras comunes (stop words)
        stop_words = set(stopwords.words('spanish'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]

        cleaned_text_list.append(filtered_text)

    # Devuelve el texto limpio
    return {'text': cleaned_text_list}

In [9]:
sentences_corpus = subset.map(clean_text, batched=True)

In [10]:
sentences_corpus['text'][:3]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía'],
 ['realidad',
  'nuevo',
  'om',
  'sólo',
  'puede',
  'mantener',
  'ilusión',
  'supremacía',
  'mágica',
  'siempre',
  'reprima',
  'desvíe',
  'potencial',
  'humano',
  'mora',
  'verdadera',
  'magia',
  'decir',
  'capacidad',
  'innata',
  'especie',
  'magia',
  'interactiva',
  'poderes',
  'animación',
  'diosa',
  'planetaria']]

## Carga y uso de modelo de embeddings Word2Vec

In [11]:
model = Word2Vec(sentences_corpus['text'], vector_size=100, window=5, min_count= 2, workers=6, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [12]:
model.wv['rey']

array([ 3.49112034e-01, -3.26905102e-01, -9.20881629e-02,  9.24129635e-02,
        2.72101551e-01,  2.16216236e-01, -2.78558172e-02,  5.16815960e-01,
       -2.09546253e-01, -2.65958011e-01, -1.16155863e+00, -2.35676065e-01,
       -2.12546647e-01,  6.98509395e-01, -7.87428617e-02,  4.79462519e-02,
       -3.44289154e-01, -3.82320076e-01, -2.13814825e-01, -1.37528986e-01,
        6.45280063e-01,  4.93542284e-01,  1.09555900e+00, -2.31672734e-01,
        6.69048131e-01,  3.93133523e-04, -3.63920517e-02, -3.03488076e-01,
       -1.77857205e-01, -5.91956861e-02,  6.95672259e-02,  6.76907480e-01,
       -2.97301915e-02, -6.52781129e-01, -4.79972064e-02,  5.90684891e-01,
       -1.23966232e-01, -3.59716207e-01, -6.53250664e-02, -8.60686362e-01,
       -4.80895750e-02, -8.88880610e-01,  5.05257428e-01,  6.87248260e-03,
       -1.18772320e-01,  1.62663877e-01, -2.38495246e-01, -4.86840457e-01,
       -1.68601453e-01,  3.73677731e-01, -4.31218892e-01, -4.06613320e-01,
        6.46060228e-01,  

In [13]:
##comida, ser, reina, television
model.wv.most_similar(['television'],topn=3)

[('entertainment', 0.8574265241622925),
 ('supren', 0.8553233742713928),
 ('gsm', 0.8523894548416138)]

In [14]:
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

In [15]:
#MMMO
display(word_vectors)
display(vectors)
display(words)

array([[ 0.02912564,  0.29023004, -0.00571955, ..., -0.4639063 ,
         0.42262894,  0.6452081 ],
       [-0.40865612,  0.24880876,  0.06341856, ..., -0.45689234,
         0.1091899 ,  0.14513676],
       [-0.49264842, -0.0562623 ,  0.11723249, ..., -0.7188871 ,
         0.2465061 , -0.2177769 ],
       ...,
       [ 0.10998932,  0.12781289,  0.02438386, ..., -0.10849639,
         0.04550839,  0.10815445],
       [ 0.12090188,  0.04213727,  0.00180553, ..., -0.05744112,
         0.01487436,  0.06404276],
       [ 0.09379203,  0.10408198,  0.05195499, ..., -0.12108329,
         0.03102642,  0.14544849]], dtype=float32)

['si',
 'puede',
 'ser',
 '“',
 '”',
 'años',
 'dios',
 'usted',
 'tiempo',
 'así',
 'vida',
 'parte',
 'mundo',
 'cada',
 'gran',
 'dos',
 'vez',
 'pueden',
 'después',
 'hacer',
 'sólo',
 'lugar',
 'personas',
 'forma',
 'día',
 'mismo',
 'todas',
 'ahora',
 'internet',
 'sido',
 'información',
 '«',
 'ciudad',
 'bien',
 '»',
 'debe',
 'iglesia',
 'tierra',
 'sistema',
 'hace',
 'aquí',
 'hecho',
 'través',
 'grupo',
 'casa',
 'agua',
 'ayuda',
 'nuevo',
 'país',
 'año',
 'general',
 'dijo',
 'cualquier',
 'unidos',
 'siempre',
 'trabajo',
 'sino',
 'además',
 'hoy',
 'mejor',
 'manera',
 'hacia',
 'medio',
 'tan',
 'menos',
 'toda',
 'cuenta',
 'embargo',
 'centro',
 'mayor',
 'tener',
 'bajo',
 'entonces',
 'ver',
 'hombre',
 'mientras',
 'ue',
 'ejemplo',
 'nueva',
 'días',
 'europa',
 'momento',
 'hotel',
 'servicio',
 'gente',
 'posible',
 'solo',
 'decir',
 'nombre',
 'incluso',
 'caso',
 'servicios',
 'importante',
 'historia',
 'desarrollo',
 'dentro',
 'tipo',
 'países',
 'p

## Almacenamiento de embeddings

In [16]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('embeddings.tsv',sep='\t', index=False)

In [17]:
df_words = pd.DataFrame(words)
df_words.to_csv('labels.tsv',sep='\t', index=False)